# step by step notebook for reproducing the workflow from RT pred manuscript 

Ensure that the requriements are met i.e.
- rdkit 
- pytorch 
- deepchem
- chemprop
- etc.. 




In [1]:
import pandas as pd 
#from rdkit import Chem
from rdkit import RDLogger 
import deepchem as dc
import os 

from functions.featurizers import get_METLIN_data, LogD_calculations, get_features
from functions.data_splitting_lib import feature_splitter_csv, feature_splitter_diskdatasets


RDLogger.DisableLog('rdApp.*') # disable rdkit log
seed = 42


Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some PyTorch models, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'torch'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


## Featurization and data splitting

This workflow is run using the METLIN SMRT Dataset published by Domingo-Almenara et al. https://www.nature.com/articles/s41467-019-13680-7 

But any dataset could be used, however, it must contain columns 'rt' and 'smiles'

By default we remove non-retained compounds from the METLIN SMRT dataset and subsample (2000, randomly)

In [2]:
get_METLIN_data(sampling = 2000, only_retained = True)

saved to:  ../data/metlin_smrt\sample_dataset_2000_datapoints.csv
                                                  smiles      rt
53863  CCCn1cc(C2=CCN(S(=O)(=O)c3c(C)nn(C)c3C)CC2)c2c...   896.5
6152                           O=C1NCCCc2c1oc1ccc(O)cc21   567.3
40962  CCN(CC)S(=O)(=O)c1ccc(Cl)c(-c2nnc(-c3ccc(OC)cc...  1091.8


There are a number of features that we use for modeling. 

- LogD descriptor set (pH 0.5 to 7.4)
- ECFP4-2048 fingerprints
- RDKit descriptors 
- molecular graph convolutions 

based on the feature_list ``` feature_list = ['logD', 'ecfp4_csv', 'ecfp4_disk', 'rdkit_csv', 'rdkit_disk', 'molgraphconv]``` it is possible to define which features to be calculated. 

Note that the ECFP4 and RDKit descriptors come in two variants: a flat .csv file or a DeepChem DiskData object. This choice will depend on downstream application and models.


OBS! LogD calculations are based on the proprietary software cxcalc from Chemaxon (https://docs.chemaxon.com/display/docs/cxcalc-command-line-tool.md). A path to a chemaxon licence is needed for this calculation, as well as a specified path to the chemaxon tool. If not provided, the LogD calculations are skipped


In [4]:
# file paths
path_to_data = '../data/metlin_smrt/sample_dataset_2000_datapoints.csv'
feature_dir = '../data/metlin_smrt/features/' 

feature_list = ['logD', 
                'ecfp4', 
                'rdkit', 
                'molgraphconv'
                ]

get_features(path_to_data, feature_dir, feature_list, path_to_chemaxon_licence= r"G:\Nuevolution\ChemAxon\licenses\license.cxl", path_to_chemaxon_tools = r"C:\nuevolution\knime_temp")

ChemAxon-based LogD calculations - to .CSV
*** NA values in: ../data/metlin_smrt/features/logd_calculations
ECFP4 Featurization - to DiskDataset
ECFP4 Featurization - to .CSV
RDKit Descriptors - to diskdataset
RDKit Descriptors - to .CSV
MolGraphConv Feat - to diskdataset


### Data Splitting 

The data is split into train/test. In addition the train dataset is split 5 times to allow for 5-fold cross validation. 

In [ ]:
#########
# Arguments 
#########

number_of_splits = 5

# random seed
seed = 42

# do global split? 
global_split = False


# base directory for all the presplit data
base_data_dir = '../../data/pub_data/premade_splits/'
if not os.path.exists(base_data_dir):
    os.makedirs(base_data_dir)

path_to_features = '../../data/pub_data/features'
path_to_splits = base_data_dir

In [ ]:

#